# Interacting with GPIO from MicroBlaze

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
import threading
import multiprocessing
import os
import socket

base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpio(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpio(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}

In [ ]:
blink = [False, False, False]
running = True
threads = []


def loop(_l, num):
    global running, blink
    while running:
        time.sleep(0.0001)
        if blink[num]:
            write_gpio(num+1, 1)
            time.sleep(1)
            write_gpio(num+1, 0)
            time.sleep(1)

def start_rgb():
    global threads
    reset_gpio()
    fork = threading.Lock()
    for i in range(3):
        t = threading.Thread(target=loop, args=(fork, i))
        threads.append(t)
        t.start()
        
def stop_rgb():
    global running, threads
    running = False
    for t in threads:
        t.join()
    reset_gpio()

In [ ]:
def run_pynq_server_C():
    global blink
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('192.168.2.99', 54321))
    sock.listen()
    conn, addr = sock.accept()
    with conn:
        print("before pynq C")
        start_rgb()
        while True:
            time.sleep(0.0001)
            data = conn.recv(1024)
            if (data.decode() == 'disconnect'):
                print("disconnect pynq C")
                stop_rgb()
                break
            elif (data.decode() == 'blue'):
                print("blue pynq C")
                blink[0] = not blink[0]
                blink[1] = False
                blink[2] = False
            elif (data.decode() == 'red'):
                print("red pynq C")
                blink[0] = False
                blink[1] = not blink[1]
                blink[2] = False
            elif (data.decode() == 'green'):
                print("green pynq C")
                blink[0] = False
                blink[1] = False
                blink[2] = not blink[2]
    print("after pynq C")

def run_pynq_client_C():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    while True:
        time.sleep(0.0001)
        if base.buttons[0].read() != 0:
            sock.connect(('192.168.2.1', 12345))
            break
    while True:
        time.sleep(1)
        if base.buttons[0].read() != 0:
            sock.sendall(b'disconnect')
            break
        elif base.buttons[1].read() != 0:
            sock.sendall(b'blue')
        elif base.buttons[2].read() != 0:
            sock.sendall(b'green') 
        elif base.buttons[3].read() != 0:
            sock.sendall(b'red')
    sock.close()

p_server = multiprocessing.Process(target=run_pynq_server_C)
p_server.start()
os.system("taskset -p -c {} {}".format(0, p_server.pid))

p_client = multiprocessing.Process(target=run_pynq_client_C)
p_client.start()
os.system("taskset -p -c {} {}".format(1, p_client.pid))

p_server.join();
p_client.join();